### Notebook for training the hand history classification


In [ ]:
# import dependencies

import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
# path selection and definition of constants

dataset = '../assets/dataset/keypoint_history.csv'
model_save_path = '../assets/models/'
tflite_save_path = '../assets/models/history_model.tflite'

NUM_CLASSES = 2
RANDOM_SEED = 42
TIME_STEPS = 20
DIMENSION = 2

# training parameters
epoche = 1000
batch_size = 128

### Dataset reading 

In [ ]:
x_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (TIME_STEPS * DIMENSION)+1)))

y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

x_train, x_test, y_train, y_test = train_test_split(x_dataset, y_dataset, train_size=0.70, random_state=RANDOM_SEED)

### Model building

In [ ]:
model = None
use_lstm = True


if use_lstm:
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(TIME_STEPS*DIMENSION, )),
        tf.keras.layers.Reshape((TIME_STEPS, DIMENSION), input_shape=(TIME_STEPS*DIMENSION, )),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(16, input_shape = [TIME_STEPS, DIMENSION]),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation = 'relu'),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')  
    ])
else:
     model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(TIME_STEPS*DIMENSION, )),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation = 'relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(24, activation = 'relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation = 'relu'),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax') 
    ])
model.summary()

In [ ]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
model_save_path+'history_model.h5',verbose=1, save_weights_only=False, save_best_only= True, mode='auto')
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

# Model compilation
model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

### Training

In [ ]:
history = model.fit(
    x_train,
    y_train,
    epochs = epoche,
    batch_size = batch_size,
    validation_data=[x_test, y_test],
    callbacks = [cp_callback, es_callback] 
)

model.save(model_save_path+'history_model.h5')
model.save(model_save_path+'history_model.h5py')

In [ ]:
# Model evaluation

val_loss, val_acc = model.evaluate(x_test, y_test, batch_size)